[View in Colaboratory](https://colab.research.google.com/github/cadurkin/Phytoplankton_Taxomony_Course/blob/master/Example_community_composition_analysis.ipynb)

# Example analysis of phytoplankton community composition

- First import your data.  
- Data should be saved as a .csv file (comma-separated values).
- import libraries needed to analyze the data.  Pandas helps to work with data matrices.  Numpy works with data arrays. Matplotlib is used for plotting.

In [0]:
!pip install scikit-bio
!git clone https://github.com/cadurkin/Phytoplankton_Taxomony_Course.git  # Download data
  
import skbio as skbio
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

data = pd.read_csv('Phytoplankton_Taxomony_Course/Example_Station_M_cellcounts.csv',infer_datetime_format=True,index_col=0)
#display the first 10 rows to make sure it looks ok
data.iloc[0:10]

<b> Organize data so that cell counts are in separate matrices from metadata

In [0]:
#separate metadata rows and columns from the raw cell counts
cell_counts_andType = data.loc[data.Parameter != 'metadata']
cell_counts = cell_counts_andType.drop('Parameter',axis=1)
#display the top part of the cell count matrix to make sure it looks correct
cell_counts.iloc[0:10]


In [0]:
#Create matrix with all the metadata
metadata_andType = data.loc[data.Parameter == 'metadata']
metadata = metadata_andType.drop('Parameter',axis=1)
metadata

## Calculate the cell concentrations
- Divide cell counts by the number of sedwick rafter squares counted to calculate cells per uL.  One square = 1 microliter of sample.
- Multiply the cells per uL by the dilution or concentration factor, whichever is appropriate.  In this dataset, highly concentrated samples were diluted.  If you settled cells into a smaller volume, you will need to use the dilution factor.  For example, 1 mL concentrated from 10 mL = 0.1 dilution factor.


In [0]:
cells_per_uL = cell_counts.divide(metadata.loc['squares_counted'], axis = 1)
cells_per_uL_dilutionCorrected = cells_per_uL.multiply(metadata.loc['Dilution_factor'], axis = 1)

In [0]:
#convert the cell concentration to cells per mL
cells_per_mL = cells_per_uL_dilutionCorrected.multiply(1000)

## Calculate Bray-Curtis dissimilarity index matrix
- using scipy library of functions
- "pdist" calculates the pairwise distance between all samples using a chosen metric.  Here, we are using the Bray-Curtis similary metric.
- the pdist function requires an array with samples as the columns and species as the rows.  So, we need to transform our species abundance matrix using the pandas function ".T".  
- For the "pdist" function to work, the species abundance matrix also needs to be converted to an "array", which does not have any labels in the rows or columns.
- later we will need our similarity matrix to be in a "square" form, instead of the condensed output that pdist provides by default.  The squareform function converts the condensed matrix into a square.
- The functions used in later steps will also require the similarity matrix to be in a specialized "distance matrix" format used by that function.  The last line of code here converts our square matrix into a distance matrix that includes the sample IDs.

In [0]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

bc_matrix_condensed = pdist(np.asarray(cells_per_mL.T),metric = 'braycurtis')
bc_matrix_square = squareform(bc_matrix_condensed)

sample_list = np.asarray(cell_counts.T.index)
bc_matrix = skbio.stats.distance.DistanceMatrix(data = bc_matrix_square,ids=sample_list)
bc_matrix

### Perform principal coordinates analysis (aka: multi-dimensional scaling analysis) with bray-curtis matrix

- PCoA attempts to place the distances from the matrix into dimensional space.
- Visualizing distances in space helps identify which groups of samples are most similar.

In [0]:
from skbio.stats import ordination
#use the special distance matrix format as the input matrix for the pcoa function (bc_matrix)
MDS = ordination.pcoa(bc_matrix)

#Plot a figure visualizing 2-D distance between samples using the first two ordinatation axes

%matplotlib inline
plt.plot(MDS.samples['PC1'],MDS.samples['PC2'], lw=0, marker = 'o')

#Add labels to the graph so you know which samples are clustering
for x in np.arange(0,len(MDS.samples['PC1'])):
    plt.text(MDS.samples.PC1[x],MDS.samples.PC2[x],sample_list[x])

## Identify clusters of samples and test whether they are significantly different
- here we are using k-means clustering.  You could also use the clustering method explained further down
- k-means clustering requires an apriori defined number of clusters to identify
- output of k-means clustering is a list of the corresponding group that each sample belongs to.

In [0]:
from sklearn.cluster import KMeans

#This function requires the square-shaped bray-curtis similarity array, without specialized formating
kmeans = KMeans(n_clusters=3, random_state=0).fit(bc_matrix_square)
kmeans.labels_

In [0]:
#test whether differences among groups are significant.
permanova_test = skbio.stats.distance.permanova(bc_matrix,kmeans.labels_)
permanova_test

from pandas import DataFrame
## Explore which species might be driving significant differences among groups of samples
- Here we use a "simper" analysis to idenify which species contribute the most to the the percent dissimilarity between two groups of samples
- We are using the simper analysis function in the ecopy library of functions
- We are using the k-means clustering results to define groups of samples
- Here, we print out the top ten species contributing to differences among groups
- Simper is not an especially robust statistical analysis, but helps point you toward the species driving differences among groups found through more robust approaches (like PERMANOVA)

In [0]:
##Here we define the simper function.  Copied from ecopy module, but edited to be compatible with pandas updates

from pandas import DataFrame

def simper(data, factor, spNames=None):
	if not isinstance(data, (DataFrame, np.ndarray)):
		msg = 'datamust be either numpy array or dataframe'
		raise ValueError(msg)
	if isinstance(data, DataFrame):
		if (data.dtypes == 'object').any():
			msg = 'DataFrame can only contain numeric values'
			raise ValueError(msg)
		X = np.array(data).astype('float')
		spNames = data.columns
	else:
		X = data.astype('float')
	if np.min(np.sum(X, axis=1))==0:
		msg = 'One row is entirely zeros, distance calculations will be meaningless'
		raise ValueError(msg)
	if (X < 0).any():
		msg = 'Matrix contains negative values'
		raise ValueError(msg)
	if spNames is None:
		spNames = np.arange(X.shape[1])
	s1 = np.array(spNames)
	g1 = np.array(factor)
	if len(g1) != X.shape[0]:
		msg = 'Factor length must equal number of rows in matrix'
		raise ValueError(msg)
	if len(s1) != X.shape[1]:
		msg = 'Species names must equal number of columns in matrix'
		raise ValueError(msg)
	groupIDs = np.unique(g1)
	print('\nComparison indices:')
	i = 0
	while i < len(groupIDs)-1:
		t1 = X[g1==groupIDs[i],:]
		j = i+1
		while j < len(groupIDs):
			t2 = X[g1==groupIDs[j],:]
			comp = '{0}-{1}'.format(groupIDs[i], groupIDs[j])
			n1 = t1.shape[0]
			n2 = t2.shape[0]
			deltaI = np.zeros((n1*n2, t1.shape[1]))
			k = 0
			while k < n1*n2:
				for idx1 in range(n1):
					for idx2 in range(n2):
						deltaI[k,:] = brayWrap(t1[idx1,:], t2[idx2,:])
						k += 1
			spMeans =np.round(deltaI.mean(axis=0), 2)
			spSds = np.round(deltaI.std(axis=0, ddof=1), 2)
			spRat = np.round(spMeans/spSds, 2)
			spPct = np.round(spMeans/spMeans.sum()*100, 2)
			tempDF = DataFrame({'sp_mean': spMeans,
						'sp_sd': spSds,
						'ratio': spRat,
						'sp_pct': spPct}, 
						index=[[comp]*len(spNames), spNames])
			tempDF.sort_values(by=['sp_pct'], axis=0, ascending=False, inplace=True)
			tempDF['cumulative'] = np.cumsum(tempDF['sp_pct'])
			tempDF = tempDF[['sp_mean', 'sp_sd', 'ratio', 'sp_pct', 'cumulative']]
			if i==0 and j==i+1:
				finalDF = tempDF
			else:
				finalDF = finalDF.append(tempDF)
			print(comp)
			j += 1
		i += 1
	return finalDF



def brayWrap(x,y):
	temp = np.array([x, y])
	sums = np.sum(temp)
	deltas = np.apply_along_axis(brayFunc, 0, temp, sums=sums)
	return deltas

def brayFunc(m, sums):
	delta = np.abs(m[0]-m[1])/sums
	return 100*delta


##Now run the simper analysis


In [0]:
species_driving_differences = simper(cells_per_mL.T,kmeans.labels_)
print(species_driving_differences.loc['0-1'][1:10])
print(species_driving_differences.loc['0-2'][1:10])
print(species_driving_differences.loc['1-2'][1:10])

##Explore how these species abundances differ among the three groups

- boxplots allow you to look at the distribution of sample values within groups
- can then use an ANOVA to test of signifiant differences in abundance of specific organisms among these groups

In [0]:
group0_samples = cell_counts.T.iloc[kmeans.labels_ == 0]
group1_samples = cell_counts.T.iloc[kmeans.labels_ == 1]
group2_samples = cell_counts.T.iloc[kmeans.labels_ == 2]

plt.subplots(figsize=(10,8))
plt.subplot(3,3,1)
plt.boxplot([group0_samples['Chaetoceros_concavicornus'],group1_samples['Chaetoceros_concavicornus'],group2_samples['Chaetoceros_concavicornus']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Chaetoceros_concavicornus')
plt.subplot(3,3,2)
plt.boxplot([group0_samples['Dinoflagellate_medium'],group1_samples['Dinoflagellate_medium'],group2_samples['Dinoflagellate_medium']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Dinoflagellate_medium')
plt.subplot(3,3,3)
plt.boxplot([group0_samples['Dinoflagellate_small'],group1_samples['Dinoflagellate_small'],group2_samples['Dinoflagellate_small']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Dinoflagellate_small')
plt.subplot(3,3,4)
plt.boxplot([group0_samples['Centric_small'],group1_samples['Centric_small'],group2_samples['Centric_small']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Centric_small')
plt.subplot(3,3,5)
plt.boxplot([group0_samples['Centric_medium'],group1_samples['Centric_medium'],group2_samples['Centric_medium']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Centric_medium')
plt.subplot(3,3,6)
plt.boxplot([group0_samples['Skeletonema'],group1_samples['Skeletonema'],group2_samples['Skeletonema']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Skeletonema')
plt.subplot(3,3,7)
plt.boxplot([group0_samples['Emeliania'],group1_samples['Emeliania'],group2_samples['Emeliania']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Emeliania')
plt.subplot(3,3,8)
plt.boxplot([group0_samples['Pseudo-nitzschia_large'],group1_samples['Pseudo-nitzschia_large'],group2_samples['Pseudo-nitzschia_large']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Pseudo-nitzschia_large')
plt.subplot(3,3,9)
plt.boxplot([group0_samples['Fragilariopsis_small_rounded'],group1_samples['Fragilariopsis_small_rounded'],group2_samples['Fragilariopsis_small_rounded']],positions=[1,2,3],widths = .75,labels = ['0','1','2'])
plt.title('Fragilariopsis_small')

## Alternative clustering approaches
- heierarchical clustering and visualization using a dendrogram

In [0]:
from scipy.cluster.hierarchy import dendrogram, linkage

#this function requires the condensed format of our bray-curtis similarity index
linkage_matrix = linkage(bc_matrix_condensed, "centroid")
plt.figure(figsize=[15,5])
dendrogram(linkage_matrix, labels=sample_list)
